# MoSDeF: A Molecular Simulation and Design Framework

The Molecular Simulation and Design Framework (MoSDeF) is a collection of open-source tools ([hosted on Github](https://github.com/mosdef-hub)) aimed at facilitating the construction and simulation of complex molecular systems - with a particular focus on the automated screening of large structural parameter spaces. All tools are written as Python packages and feature a Python-based API.

## Basic Foyer tutorial

The second of the MoSDeF tools we will explore is the [Foyer package](https://github.com/mosdef-hub/foyer) which provides a means for the automated application and dissemination of forcefields. This tool can be used to take a molecular model featuring particle positions and bonds (such as an mBuild `Compound`) and automatically perform atomtyping (as well as parameterization) as directed by a user-specified forcefield. In this tutorial, we'll demonstrate how Foyer works by using it to perform atomtyping and parameterization of several alkane molecules.

### Classical Force fields
Classical force fields, which describe the interaction between atoms, typically contain numerous fitting parameters, allowing them to be tuned to describe the behavior of atoms in various chemical environments.  For example, consider carbon in various local bonded environments: 
- As a terminal methyl group in an alkane, i.e., C-CH3
- As a terminal fluorinated methyl group, i.e., C-CF3
- As a terminal group in an alkene, i.e., C=CH2

Even though the element does not change (i.e., we are dealing with carbon in each case), the force field fitting parameters (e.g., Lennard-Jones epsilon and partial charges) will be different in most force fields.  As such, force field databases may contain 10s or 100s of parameters for a given element, where each set of parameters is referred to as an 'atom-type', defining the element in a different chemical context.

Properly determining the correct set of parameters for a given element (i.e., identifying which atom-type should be used), can be a difficult and error-prone task, but essential for the final results of a simulation to be correct.   

Foyer has been developed as both a scheme to encode these rules for atom-typing along with a software that evaluates these rules. Here we will first focus on the scheme to encode rules and the associated file format. 

### Foyer XML format

Foyer forcefields are defined within an XML file that contains both the 'rules' required for atomtyping as well as the forcefield parameters within a single file. The Foyer XML format is an extension of the [OpenMM forcefield XML format](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields), which you may already be familiar with. The only differences reside in the `AtomTypes` section, where several additional attributes are available (which we will examine in a moment) for each `Type` to allow for atomtyping.

Let's take a look at the basic structure of a Foyer XML file.

#### AtomTypes

In [3]:
! sed -n 2,18p OPLSaa_alkanes.xml

    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760" overrides="opls_136"/>

        <Type name="opls_136" def="[C;X4](H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760" overrides="opls_135,opls_136"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>


The `AtomTypes` section of the Foyer XML is similar to that used for OpenMM forcefield XMLs; however, each `Type` in Foyer XML supports four additional attributes not found in OpenMM:
* `def` - SMARTS string describing the chemical substructure of this atomtype (Follow [this link](https://github.com/mosdef-hub/foyer/blob/master/docs/smarts.md) for more on SMARTS-based atomtyping using Foyer.)
* `desc` - Brief description of the atomtype
* `doi` - DOI reference for parameters associated with this atomtype
* `overrides` - One or more atomtypes to 'override', providing precedence to this atomtype

Atomtypes are defined using [SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html), which provide a language for describing chemical structures and substructures.

#### Forcefield parameters

The remaining sections of the Foyer XML are currently the same as defined in the OpenMM format and feature both bonded and nonbonded forcefield parameters.

In [19]:
! sed -n 19,42p oplsaa_alkanes.xml

    <HarmonicBondForce>
        <Bond class1="CT" class2="CT" length="0.1529" k="224262.4"/>
        <Bond class1="CT" class2="HC" length="0.1090" k="284512.0"/>
    </HarmonicBondForce>
    <HarmonicAngleForce>
        <Angle class1="CT" class2="CT" class3="CT" angle="1.966986067" k="488.273"/>
        <Angle class1="CT" class2="CT" class3="HC" angle="1.932079482" k="313.800"/>
        <Angle class1="HC" class2="CT" class3="HC" angle="1.881464934" k="276.144"/>
    </HarmonicAngleForce>
    <RBTorsionForce>
        <Proper class1="CT" class2="CT" class3="CT" class4="CT" c0="2.9288"
                c1="-1.4644" c2="0.2092" c3="-1.6736" c4="0.0" c5="0.0"/>
        <Proper class1="CT" class2="CT" class3="CT" class4="HC" c0="0.6276"
                c1="1.8828" c2="0.0" c3="-2.5104" c4="0.0" c5="0.0"/>
        <Proper class1="HC" class2="CT" class3="CT" class4="HC" c0="0.6276"
                c1="1.8828" c2="0.0" c3="-2.5104" c4="0.0" c5="0.0"/>
    </RBTorsionForce>
    <NonbondedForce co

### Defining [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html)
Focusing first on atom type `opls_140`, the SMARTS string, `def="H[C;X4]"`, states that this atom-type applies when:
- The element is hydrogen, i.e., `H`
- When that hydrogen is connected to a carbon that has 4 neighbors, i.e., `[C;X4]`

Similarly, for atom type `opls_138`, the SMARTS string, `def="[C;X4](H)(H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- 4 of those neighbors are hydrogens, i.e., `(H)(H)(H)H`

For atom type `opls_136`, the SMARTS string, `def="[C;X4](H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 2 of those neighbors are hydrogens, i.e., `(H)H`

For atom type `opls_135`, the SMARTS string, `def="[C;X4](H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 3 of those neighbors are hydrogens, i.e., `(H)(H)H`

### Atomtyping using SMARTS
Let us now consider using these rules to atom-type the carbon in methane (i.e., CH4).

- `opls_138` would obviously evaluate to `True`, as it is defined for a carbon, with 4 hydrogen neighbors. 
- `opls_135` and `opls_136` would also evaulate to `True`.  In the case of opls_135, our definition only states that at least 3 of the neighbors are hydrogen, and have not made any specific claims about the identity of the 4th neighbor; similarly, for opls_136, we have only stated that 2 neighbors must be hydrogen and not specified the identity of the other 2 neighbors. 

This is an important feature of Foyer to take note of.  Foyer will evaluate all the rules in a forcefield file, rather than just stopping at the first rule that evaluates to `True`. This allows rules to be defined in any order.  Furthermore, Foyer iterates over the rules, which allows recursive definitions of usage, i.e., referring to specific atom-types in the SMARTS string. 

### Overrides
We will discuss two ways to address this. One approach is to employ `overrides`.  `overrides` provide a means to dictate rule precedence (i.e., which rules are more specific than others).  In the force field file above, `opls_138` has defined: `overrides="opls_135,opls_136"`.  That is, if `opls_138` evaluates to `True`, then it takes precedence over `opls_135` and `opls_136`, even if they evaluate to `True`. 

Similarly, if opls_136 evaluates to `True`, it `overrides="opls_136"`, thus taking precedence. 

`overrides` are especially useful if the chemical context of two different atom-types are effectively the same.  E.g., the terminal methyl group in an alkane has the same first neighbor environment as the methyl group in toluene, however the parameters (and thus atom-type) are different. Thus `overrides` can be used to state that if the more specific toluene rule evaluates to `True` it should take precedences over the more general alkane rule (as shown below):

<img src="../graphics/ch3-toluene.png" alt="Drawing" style="width: 700px;"/>

### Better SMARTS definitions
In many cases, `overrides` can be avoided by simply providing more specific SMARTS strings.  For example, the rules for `opls_135`, `opls_136`, and `opls_138` above can be made more specific by stating the identify of the other neighbors besides carbon and thus eliminate the need for `overrides`, as shown below. 

In [5]:
! sed -n 2,18p OPLSaa_alkanes2.xml

    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)(H)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760"/>

        <Type name="opls_136" def="[C;X4](H)(H)(C)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>


### Applying a force field to atomtype a system
With a force field file defined, we will apply it to an alkane system.  Here, we will use the routines previously discussed in the mBuild tutorial to construct a hexane, then perform atom-typing as part of saving our configuration to a desired file format.



In [36]:
import foyer 
from foyer import Forcefield

import mbuild 

compound = mb.load("CCC", smiles=True)
oplsaa_alkane = foyer.Forcefield("oplsaa_alkanes.xml")
typed = oplsaa_alkane.apply(compound)

# Or just call the save function

## SMARTS for Non-Atomistic Systems


Thus far, all of our tutorials for mBuild and Foyer have focused on fully atomistic systems, however, MoSDeF is not limited to such systems. 

First, we will demonstrate how to construct a generic coarse-grained polymer system using mBuild, then demonstrate how to define an associated forcefield file.

As done previous in the mBuild tutorials, we will create `Compounds` to describe repeat units in our polymer. Here we will create two `Compounds`, one to describe the central beads (`_A`) and one for the terminal groups (`_B`).

Note, to properly handle non-atomistic types, Foyer expects names to be prefaced by an underscore (e.g., `_A`,`_B`, `_CH2`, etc. ).

In [32]:
class CentralBead(mb.Compound):
    def __init__(self):
        super(CentralBead, self).__init__()
        
        bead = mb.Particle(pos=[0.0, 0.0, 0.0], name='_A')
        self.add(bead)
        up_port = mb.Port(anchor=bead, orientation=[0, 0, 1], separation=0.05)
        down_port = mb.Port(anchor=bead, orientation=[0, 0, -1], separation=0.05)
        self.add(up_port, label='up')
        self.add(down_port, label='down')
        
class TerminalBead(mb.Compound):
    def __init__(self):
        super(TerminalBead, self).__init__()
        
        bead = mb.Particle(pos=[0.0, 0.0, 0.0], name='_B')
        self.add(bead)

        cap_port = mb.Port(anchor=bead, orientation=[0, 0, 1], separation=0.05)
        self.add(cap_port, label='cap')

class CGPolymer(mb.Compound):
    def __init__(self, chain_length):
        super(CGPolymer, self).__init__()
        
        terminal_bead = TerminalBead()
        last_unit = CentralBead()
        mb.force_overlap(move_this=terminal_bead,
                         from_positions=terminal_bead['cap'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='_A[$]')
        self.add(terminal_bead, label='up-cap')   
        for _ in range(chain_length - 3):
            current_unit = CentralBead()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='_A[$]')
            last_unit=current_unit
        terminal_bead = TerminalBead()
        mb.force_overlap(move_this=terminal_bead,
                         from_positions=terminal_bead['cap'],
                         to_positions=last_unit['down'])
        self.add(terminal_bead, label='down-cap')
        if chain_length < 3:
            print("Note, the shortest chain this function will make is 3")
cg_polymer = CGPolymer(chain_length=6)
cg_polymer.visualize()

/Users/quachcd/Documents/dev/env/mbuild/mbuild/conversion.py:1752: UserWarning: No element attribute associated with '<_A pos=([0. 0. 0.]), 2 bonds, id: 6130221120>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/Documents/dev/env/mbuild/mbuild/conversion.py:1752: UserWarning: No element attribute associated with '<_B pos=([-8.9071e-18 -6.2221e-19  1.0000e-01]), 1 bonds, id: 6130224288>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

With a simple mBuild code to create CG polymers, let us examine how to define a forcefield file.

In this forcefield, we will assume all particles have the same bonds, angles, and Lennard-Jones sigma, regardless of type, however, we will modify Lennard-Jones epsilon based on chemical context. 

Here we will define the forcefield such that:
- when bead `_A` has 2 bonded neighbors of type `_A`, epsilon = 1.0 
 - atom-type: `cg_A_AA`
 - SMARTS definition: `[_A;X2](_A)(_A)`
- when bead `_A` has 1 bonded neighbor of type `_A` and one of type `_B`, epsilon = 1.25 
 - atom-type `cg_A_AB`
 - SMARTS definition: `[_A;X2](_A)(_B)`
- when bead `_A` has 2 bonded neighbors of type `_B`  epsilon = 1.5
 - atom-type `cg_A_BB`
 - SMARTS definition: `[_A;X2](_B)(_B)`
- when bead `_B` has any neighbor, epsilon = 2.0 regardless of who it is bonded 
 - `atom-type cg_B`
 - SMARTS definition: `[_B;X1]`

The xml forcefield file is shown below:

In [35]:
cat utils/CG_polymer.xml


cat: utils/CG_polymer.xml: No such file or directory


### Foyer + mBuild

As we saw in the mBuild tutorial, Foyer has been designed to integrate seamlessly with mBuild, although it can also be used as a standalone package. To call Foyer from mBuild all a user needs to do is specify the forcefield XML file to use by passing either the `forcefield_name` or `forcefield_files` flag to `Compound.save()`. Internally this will perform parameterization of the mBuild `Compound` using Foyer.

Additionally, the `references_file` flag can be used to output a file (in BibTeX format) containing the references for the parameters used for parameterization of this `Compound`.

Here, we explore this functionality by loading a simple methane molecule into an mBuild `Compound` and save using the forcefield we've just defined.

In [25]:
import mbuild as mb

ch4 = mb.load("C", smiles=True)
ch4.save("CH4.top", forcefield_files='oplsaa_alkanes.xml', foyer_kwargs={"references_file":"ch4.bib"}, overwrite=True)


/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:629: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:641: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:653: UserWarning: No combining rule found in force field XML file.
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 4, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


In [26]:
help(mb.Compound.save)

Help on function save in module mbuild.compound:

save(self, filename, show_ports=False, forcefield_name=None, forcefield_files=None, forcefield_debug=False, box=None, overwrite=False, residues=None, combining_rule='lorentz', foyer_kwargs=None, **kwargs)
    Save the Compound to a file.
    
    Parameters
    ----------
    filename : str
        Filesystem path in which to save the trajectory. The extension or
        prefix will be parsed and control the format. Supported extensions:
        'hoomdxml', 'gsd', 'gro', 'top', 'lammps', 'lmp', 'mcf'
    show_ports : bool, optional, default=False
        Save ports contained within the compound.
    forcefield_files : str, optional, default=None
        Apply a forcefield to the output file using a forcefield provided
        by the `foyer` package.
    forcefield_name : str, optional, default=None
        Apply a named forcefield to the output file using the `foyer`
        package, e.g. 'oplsaa'. `Foyer forcefields
        <https://gi

In [27]:
! cat ch4.bib

@article{Jorgensen_1996,
	doi = {10.1021/ja9621760},
	url = {https://doi.org/10.1021%2Fja9621760},
	year = 1996,
	month = {nov},
	publisher = {American Chemical Society ({ACS})},
	volume = {118},
	number = {45},
	pages = {11225--11236},
	author = {William L. Jorgensen and David S. Maxwell and Julian Tirado-Rives},
	title = {Development and Testing of the {OPLS} All-Atom Force Field on Conformational Energetics and Properties of Organic Liquids},
	journal = {Journal of the American Chemical Society},
	note = {Parameters for atom types: opls_138, opls_140}
}


## Quick-start

If you are interested in defining your existing forcefield(s) in the Foyer XML format, we currently have a forcefield template hosted on Github to help you get started - https://github.com/mosdef-hub/forcefield_template, that includes some basic scripts for validation and error checking. We also refer interested users to the [OpenMM documentation](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields) for detailed instructions on forcefield creation.